In [1]:
# Needed to convert the Hamiltonian into pauli gates
from qiskit.quantum_info import SparsePauliOp

#Packages for numerical calculation
import statistics
import numpy as np

import scipy as sp

#For eigenvalues and eigenvectors
from numpy import linalg

from scipy.sparse import lil_matrix

In [2]:
# Parameters for the construction of the Hamiltonian
N_qubits = 2                           # Chose the number of qubits

# Function to display a list as a string in a compact format, ensuring it doesn't break across multiple lines
def display_nicely_list(list):
    #set the number of decimals retained
    decimals = 6
    zero_chop_limit = 1e-8
   
    return "[ " + ", ".join("0" if x == 0 or abs(x)<= zero_chop_limit else f"{x:.{decimals}f}" for x in list) + " ]"

#======================================================================================================================

def sparse_ghz_state(n_qubits):

    # Size of the state
    state_size = 2**n_qubits
   
    # Create a sparse state_vector with only non-zero elements
    # Create an empty sparse row vector
    state_vector = lil_matrix((1, state_size))  
   
    # Set the amplitudes for non-zero element
    state_vector[0, 0] = 1/np.sqrt(2)  # |000...0>
    state_vector[0, state_size-1] = 1/np.sqrt(2)  # |111...1>
   
    return state_vector
#======================================================================================================================

# Hamiltonian having the bell state |Φ+> = 1 / np.sqrt(2) [|0...0> + |1...1>]  as ground state  

# Identity operator for a N-qubit system
I_n_qubit =sp.sparse.eye(2**N_qubits)

Hbell = I_n_qubit.toarray() -2 * np.outer(sparse_ghz_state(N_qubits).toarray(), np.conj(sparse_ghz_state(N_qubits)).toarray())

Hbell_op = SparsePauliOp.from_operator(Hbell, atol = 1e-8)

N_qubits = Hbell_op.num_qubits

N_paulis = Hbell_op.size

print(Hbell_op)
print("Number of qubits: ", N_qubits)
print("Number of pauli strings: ", N_paulis)

SparsePauliOp(['II', 'XX', 'YY', 'ZZ'],
              coeffs=[ 0.5+0.j, -0.5+0.j,  0.5-0.j, -0.5+0.j])
Number of qubits:  2
Number of pauli strings:  4


In [3]:
# Calculate the eigenvalue and eigenvetor and save the min eigenvalue
eigenvalues, eigenVectors = np.linalg.eig(Hbell)
   
# Sort eigenvalues and eigenvectors
# Sort eigenvalues indices in ascending order (min to max) [for descending use np.argsort(eigenvalues)[::-1] ]
sorted_idx = np.argsort(eigenvalues)

sorted_eigenvalues = eigenvalues.real[sorted_idx]
#sorting the eigenVectors by reordering the columns of eigenvectors array
sorted_eigenVectors = eigenVectors.real[:, sorted_idx]  

#keep in mind that the eigenvectors are the columns of eigenVectors array
ground_state = sorted_eigenVectors[:,0]

#Print the first n_energy_states exact eigenstates
print("Eigenvalue and eigenvector")
for i in range (4):
    print(sorted_eigenvalues[i]," " + display_nicely_list(sorted_eigenVectors[:,i]))

Eigenvalue and eigenvector
-0.9999999999999996  [ 0.707107, 0, 0, 0.707107 ]
1.0  [ 0.707107, 0, 0, -0.707107 ]
1.0  [ 0, 1.000000, 0, 0 ]
1.0  [ 0, 0, 1.000000, 0 ]


In [14]:
eigenVectors

array([[ 0.70710678,  0.70710678,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ],
       [-0.70710678,  0.70710678,  0.        ,  0.        ]])

In [12]:
psi = sorted_eigenVectors[0].reshape(2, 2)
rho_A = psi @ psi.conjugate().T
print("Reduced density matrix for system A:\n", rho_A)
vals, _ = np.linalg.eig(rho_A)
# Ensure numerical stability by considering only positive eigenvalues.
vals = vals[vals > 0]
entropy = -np.sum(vals * np.log(vals))
print(entropy)


print(psi)

U, s, Vh = np.linalg.svd(psi, full_matrices=False)
print(U)
print(s)
print(Vh)

p = s**2

print(p)
entropy = -np.sum(p[p > 0] * np.log2(p[p > 0]))
print("Entanglement entropy:", entropy)


Reduced density matrix for system A:
 [[1. 0.]
 [0. 0.]]
2.220446049250313e-16
[[0.70710678 0.70710678]
 [0.         0.        ]]
[[1. 0.]
 [0. 1.]]
[1. 0.]
[[ 0.70710678  0.70710678]
 [-0.70710678  0.70710678]]
[1. 0.]
Entanglement entropy: -0.0
